# 05-01 동영상 안전하게 로드

## 이전 동영상 플레이

In [2]:
import cv2
import numpy as np

### 동영상 재생

In [ ]:
capture = cv2.VideoCapture('./data/file_example_MP4_640_3MG.mp4')

while True:
    ret, frame = capture.read()
    if not ret: break

    cv2.imshow("video", frame)
    if cv2.waitKey(1) == ord('q'): break

capture.release()
cv2.destroyAllWindows()

## 안전 방식

In [ ]:
import cv2, time, os
import numpy as np

VIDEO_PATH = './data/file_example_MP4_640_3MG.mp4'  # 지구 자전 동영상

# 백엔드(FFMPEG) 우선 시도 → 실패 시 자동 대체
cap = cv2.VideoCapture(VIDEO_PATH, cv2.CAP_FFMPEG)
if not cap.isOpened():
    cap = cv2.VideoCapture(VIDEO_PATH)  # CAP_ANY

In [67]:
if not cap.isOpened():
    raise RuntimeError('동영상을 열 수 없습니다. 경로/코덱을 확인하세요.')  # 동영상 로드 실패시 예외발생

In [68]:
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps) if fps and fps > 1 else 33  # 안전 기본값 30fps

In [69]:
paused = False
while True:
    if not paused:
        ret, frame = cap.read()
        if not ret:
            print('끝까지 재생했거나 프레임 읽기 실패. 종료합니다.')
            break

        # 화면 우상단에 프레임 번호/총프레임 표시
        idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cv2.putText(frame, f"{idx}/{total}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,255), 2)
        cv2.putText(frame, f"FPS : {fps}", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 2)

        cv2.imshow('Video Player', frame)

    key = cv2.waitKey(0 if paused else delay) & 0xFF

    if key in (ord('q'), 27):  # q or ESC
        break
    elif key == ord(' '):      # Space → pause/resume
        paused = not paused

cap.release()
cv2.destroyAllWindows()

끝까지 재생했거나 프레임 읽기 실패. 종료합니다.


### 전체 소스

In [ ]:
import cv2, time, os
import numpy as np

VIDEO_PATH = './data/file_example_MP4_640_3MG.mp4' # 지구 자전 동영상

# 1. 백엔드(FFMPEG) 우선 시도 → 실패 시 자동 대체
cap = cv2.VideoCapture(VIDEO_PATH, cv2.CAP_FFMPEG)  
if not cap.isOpened():
    cap = cv2.VideoCapture(VIDEO_PATH) # CAP_ANY

if not cap.isOpened():
    raise RuntimeError('동영상을 열 수 없습니다. 경로/코덱을 확인하세요.') # 동영상 로드 실패시 예외발생

fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps) if fps and fps > 1 else 33   # 2. 지연시간 안전 기본값 30fps

paused = False
while True:
    if not paused:
        ret, frame = cap.read()   # 3. ret 기반 루프
        if not ret:
            print('끝까지 재생했거나 프레임 읽기 실패. 종료합니다.')
            break

        # 화면 우상단에 프레임 번호/총프레임 표시
        idx = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cv2.putText(frame, f"{idx}/{total}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,255), 2)
        cv2.putText(frame, f"FPS : {fps}", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 2)
        cv2.imshow('Video Player', frame)

    key = cv2.waitKey(0 if paused else delay) & 0xFF
    if key in (ord('q'), 27):   # q or ESC
        break
    elif key == ord(' '):       # Space → pause/resume
        paused = not paused
    
    elif key == ord('s') and 'frame' in locals():
        # 유니코드 경로에서도 안전 저장 (imencode→tofile)
        ok, buf = cv2.imencode('.png', frame)
        if ok:
            fname = f'./result/snap_{int(time.time())}.png'
            buf.tofile(fname)
            print('스크린샷 저장:', fname)
    elif key == 81 or key == ord('h'):  # ← (또는 h)
        # 1초 뒤로
        cur = cap.get(cv2.CAP_PROP_POS_MSEC)
        cap.set(cv2.CAP_PROP_POS_MSEC, max(cur - 1000, 0))
        paused = False
    elif key == 83 or key == ord('l'):  # → (또는 l)
        # 1초 앞으로
        cur = cap.get(cv2.CAP_PROP_POS_MSEC)
        cap.set(cv2.CAP_PROP_POS_MSEC, cur + 1000)
        paused = False

cap.release()
cv2.destroyAllWindows()